In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import warnings
import seaborn as sns
import numpy as np

# SQL Alchemy
from sqlalchemy import create_engine, event
import time
from datetime import datetime

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name
from config import remote_db_user, remote_db_pwd, remote_db_endpoint, remote_db_port, remote_db_name

warnings.filterwarnings('ignore')

In [ ]:
#import our CSV file
yearly_stats_all_pos_df = pd.read_csv('yearly data/all_players_yearly_stats_2002-2020.csv')
yearly_stats_all_pos_df.head()

In [ ]:
yearly_stats_all_pos_df.drop(['Rk', '2pt Convs Made', '2pt Conv Passes', 'FPts', 
       'DK Fpts', 'FD Fpts', 'VBD', 'Pos Rank', 'Overall Rank', 'Fumbles', 'Games Started'], axis=1, inplace=True)

In [ ]:
yearly_stats_all_pos_df.columns

In [ ]:
#yearly_stats_all_pos_df['FantasyPoints'] = yearly_stats_all_pos_df['Rush Yds']*0.1 + yearly_stats_all_pos_df['Rush TD']*6 + yearly_stats_all_pos_df['Receptions'] + yearly_stats_all_pos_df['Rec Yds']*0.1 + yearly_stats_all_pos_df['Rec TD']*6 - yearly_stats_all_pos_df['Fumbles Lost']*2 + yearly_stats_all_pos_df['Pass Yds']*0.04 + yearly_stats_all_pos_df['Pass TD']*4

In [ ]:
yearly_stats_all_pos_df['FantasyPoints'] = yearly_stats_all_pos_df['PPR']

In [ ]:
#Create new column for Fantasy points per game.
yearly_stats_all_pos_df['FantasyPoints/Game'] = yearly_stats_all_pos_df['FantasyPoints']/yearly_stats_all_pos_df['Games Played']
yearly_stats_all_pos_df['FantasyPoints/Game'] = yearly_stats_all_pos_df['FantasyPoints/Game'].apply(lambda x: round(x, 2))

In [ ]:
yearly_stats_all_pos_df = yearly_stats_all_pos_df[yearly_stats_all_pos_df['Season'] == 2020]

In [ ]:
#separate dataframes based off position
rb_df = yearly_stats_all_pos_df[yearly_stats_all_pos_df['Pos'] == 'RB']
qb_df = yearly_stats_all_pos_df[yearly_stats_all_pos_df['Pos'] == 'QB']
wr_df = yearly_stats_all_pos_df[yearly_stats_all_pos_df['Pos'] == 'WR']
te_df = yearly_stats_all_pos_df[yearly_stats_all_pos_df['Pos'] == 'TE']

# Grab Headshots

In [ ]:
#bring in roster data
roster_df = pd.read_csv('roster_2020.csv')

#change the player name column to have the same format as our passing_df one
#roster_df['receiver_player_name'] = roster_df['teamPlayers.displayName'].apply(lambda x: '.'.join([x.split()[0][0], x.split()[-1]]))

#filter our results a bit more so we get closer to our solution
roster_df = roster_df.loc[roster_df['position'].isin(['QB', 'RB', 'WR', 'TE']), ['full_name', 'headshot_url']]

#reset the index to a range index 0 -> len(roster_df)
roster_df = roster_df.reset_index(drop=True)

roster_df

# RB: Compare usage vs efficiency 

In [ ]:
#Create new column for usage per game. Usage is defined as # of targets + carries
rb_df['Usage/Game'] = (rb_df['Rush Att'] + rb_df['Rec Tgt'])/rb_df['Games Played']

#round each row value to two decimal places
rb_df['Usage/Game'] = rb_df['Usage/Game'].apply(lambda x: round(x, 2))

rb_df['Usage'] = rb_df['Rec Tgt'] + rb_df['Rush Att'] # create a new column for usage
rb_df['UsageRank'] = rb_df['Usage'].rank(ascending=False)
rb_df['FantasyPointsRank'] = rb_df['FantasyPoints'].rank(ascending=False)

In [ ]:
#just for styling. We imported seaborn earlier as sns.
sns.set_style('whitegrid')

#create a canvas with matplotlib
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#basic regression scatter plot with trendline
plot = sns.regplot(
x=rb_df['Usage/Game'],
y=rb_df['FantasyPoints/Game'],
scatter=True,)

plt.tight_layout();
plt.savefig('../static/images/RB_usage_per_game', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)


#How does efficiency correlate to fantasy football performance?
rb_df['TD/Usage'] = (rb_df['Rush TD']+ rb_df['Rec TD'])/(rb_df['Rush Att'] + rb_df['Rec Tgt'])
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#Make sure there is an adequete sample size
rb_df = rb_df[rb_df['Rush Att'] > 20]
plot = sns.regplot(
x=rb_df['TD/Usage'],
y=rb_df['FantasyPoints/Game'],
scatter=True)

plt.tight_layout();
plt.savefig('../static/images/RB_TD_per_usage', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
from sklearn.model_selection import train_test_split
X = rb_df['Usage'].values.reshape(-1, 1)
Y = rb_df['FantasyPoints'].values.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm
y_pred = regressor.predict(x_test)

In [ ]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head()

In [ ]:
rb_df['FantasyPointPrediction'] = regressor.predict(X) # create a new column for usage

#round each row value to two decimal places
rb_df['FantasyPointPrediction'] = rb_df['FantasyPointPrediction'].apply(lambda x: round(x, 1))

In [ ]:
rb_df_with_headshots = rb_df.merge(roster_df, left_on='Player', right_on='full_name', how='left')
rb_df_with_headshots = rb_df_with_headshots.sort_values(by='UsageRank')   

#reset the index to a range index 0 -> length
rb_df_with_headshots = rb_df_with_headshots.reset_index(drop=True)

rb_df_with_headshots.head(10) 

# WR: Compare usage vs efficiency

In [ ]:
#Create new column for usage per game. Usage is defined as # of targets + carries
wr_df['Usage/Game'] = (wr_df['Rush Att'] + wr_df['Rec Tgt'])/wr_df['Games Played']

#round each row value to two decimal places
wr_df['Usage/Game'] = wr_df['Usage/Game'].apply(lambda x: round(x, 2))

wr_df['Usage'] = wr_df['Rec Tgt'] + wr_df['Rush Att'] # create a new column for usage
wr_df['UsageRank'] = wr_df['Usage'].rank(ascending=False)
wr_df['FantasyPointsRank'] = wr_df['FantasyPoints'].rank(ascending=False)

In [ ]:
#just for styling. We imported seaborn earlier as sns.
sns.set_style('whitegrid')

#create a canvas with matplotlib
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#basic regression scatter plot with trendline
plot = sns.regplot(
x=wr_df['Usage/Game'],
y=wr_df['FantasyPoints/Game'],
scatter=True,)

plt.tight_layout();
plt.savefig('../static/images/WR_usage_per_game', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)



#How does efficiency correlate to fantasy football performance?
wr_df['TD/Usage'] = (wr_df['Rush TD']+ wr_df['Rec TD'])/(wr_df['Rush Att'] + wr_df['Rec Tgt'])
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#Make sure there is an adequete sample size
wr_df = wr_df[wr_df['Receptions'] > 20]
plot = sns.regplot(
x=wr_df['TD/Usage'],
y=wr_df['FantasyPoints/Game'],
scatter=True)

plt.tight_layout();
plt.savefig('../static/images/WR_TD_per_usage', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
from sklearn.model_selection import train_test_split
X = wr_df['Usage'].values.reshape(-1, 1)
Y = wr_df['FantasyPoints'].values.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm
y_pred = regressor.predict(x_test)

df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head()

In [ ]:
wr_df['FantasyPointPrediction'] = regressor.predict(X) # create a new column for usage

wr_df['FantasyPointPrediction'] = wr_df['FantasyPointPrediction'].apply(lambda x: round(x, 1))

In [ ]:
wr_df_with_headshots = wr_df.merge(roster_df, left_on='Player', right_on='full_name', how='left')
wr_df_with_headshots = wr_df_with_headshots.sort_values(by='UsageRank')  

#reset the index to a range index 0 -> length
wr_df_with_headshots = wr_df_with_headshots.reset_index(drop=True)

wr_df_with_headshots.head(10)  

# TE: Compare usage vs efficiency


In [ ]:
#Create new column for usage per game. Usage is defined as # of targets + carries
te_df['Usage/Game'] = (te_df['Rush Att'] + te_df['Rec Tgt'])/te_df['Games Played']

#round each row value to two decimal places
te_df['Usage/Game'] = te_df['Usage/Game'].apply(lambda x: round(x, 2))

te_df['Usage'] = te_df['Rec Tgt'] + te_df['Rush Att'] # create a new column for usage
te_df['UsageRank'] = te_df['Usage'].rank(ascending=False)
te_df['FantasyPointsRank'] = te_df['FantasyPoints'].rank(ascending=False)

In [ ]:
#just for styling. We imported seaborn earlier as sns.
sns.set_style('whitegrid')

#create a canvas with matplotlib
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#basic regression scatter plot with trendline
plot = sns.regplot(
x=te_df['Usage/Game'],
y=te_df['FantasyPoints/Game'],
scatter=True,)

plt.tight_layout();
plt.savefig('../static/images/TE_usage_per_game', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)



#How does efficiency correlate to fantasy football performance?
te_df['TD/Usage'] = (te_df['Rush TD']+ te_df['Rec TD'])/(te_df['Rush Att'] + te_df['Rec Tgt'])
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#Make sure there is an adequete sample size
te_df = te_df[te_df['Receptions'] > 10]
plot = sns.regplot(
x=te_df['TD/Usage'],
y=te_df['FantasyPoints/Game'],
scatter=True)

plt.tight_layout();
plt.savefig('../static/images/TE_TD_per_usage', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
from sklearn.model_selection import train_test_split
X = te_df['Usage'].values.reshape(-1, 1)
Y = te_df['FantasyPoints'].values.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm
y_pred = regressor.predict(x_test)

df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head()

In [ ]:
te_df['FantasyPointPrediction'] = regressor.predict(X) # create a new column for usage

te_df['FantasyPointPrediction'] = te_df['FantasyPointPrediction'].apply(lambda x: round(x, 1))

In [ ]:
te_df_with_headshots = te_df.merge(roster_df, left_on='Player', right_on='full_name', how='left')
te_df_with_headshots = te_df_with_headshots.sort_values(by='UsageRank') 

#reset the index to a range index 0 -> length
te_df_with_headshots = te_df_with_headshots.reset_index(drop=True)

te_df_with_headshots.head(10) 

# QB: Compare usage vs efficiency


In [ ]:
yearly_stats_all_pos_df.columns

In [ ]:
#Create new column for usage per game. Usage is defined as # of targets + carries
qb_df['Usage/Game'] = (qb_df['Pass Att'] + qb_df['Rush Att'])/qb_df['Games Played']

#round each row value to two decimal places
qb_df['Usage/Game'] = qb_df['Usage/Game'].apply(lambda x: round(x, 2))

qb_df['Usage'] = qb_df['Pass Att'] + qb_df['Rush Att'] # create a new column for usage
qb_df['UsageRank'] = qb_df['Usage'].rank(ascending=False)
qb_df['FantasyPointsRank'] = qb_df['FantasyPoints'].rank(ascending=False)

In [ ]:
qb_df

In [ ]:
#just for styling. We imported seaborn earlier as sns.
sns.set_style('whitegrid')

#create a canvas with matplotlib
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#basic regression scatter plot with trendline
plot = sns.regplot(
x=qb_df['Usage/Game'],
y=qb_df['FantasyPoints/Game'],
scatter=True,)

plt.tight_layout();
plt.savefig('../static/images/QB_usage_per_game', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)



#How does efficiency correlate to fantasy football performance?
qb_df['TD/Usage'] = (qb_df['Pass TD']+ qb_df['Rush TD'])/(qb_df['Pass Att'] + qb_df['Rush Att'])
fig, ax = plt.subplots()
fig.set_size_inches(5,5)

#Make sure there is an adequete sample size
#qb_df = qb_df[qb_df['Receptions'] > 10]
plot = sns.regplot(
x=qb_df['TD/Usage'],
y=qb_df['FantasyPoints/Game'],
scatter=True)

plt.tight_layout();
plt.savefig('../static/images/QB_TD_per_usage', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
from sklearn.model_selection import train_test_split
X = qb_df['Usage'].values.reshape(-1, 1)
Y = qb_df['FantasyPoints'].values.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm
y_pred = regressor.predict(x_test)

df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head()

In [ ]:
qb_df['FantasyPointPrediction'] = regressor.predict(X) # create a new column for usage

qb_df['FantasyPointPrediction'] = qb_df['FantasyPointPrediction'].apply(lambda x: round(x, 1))

In [ ]:
qb_df_with_headshots = qb_df.merge(roster_df, left_on='Player', right_on='full_name', how='left')
qb_df_with_headshots = qb_df_with_headshots.sort_values(by='UsageRank')   

#reset the index to a range index 0 -> length
qb_df_with_headshots = qb_df_with_headshots.reset_index(drop=True)

qb_df_with_headshots.head(10) 

# Add to MySQL

In [6]:
# Create Engine and Pass in MySQL Connection

# Remote MySQL DB Connection  
remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
remote_conn = remote_engine.connect()

# Create remote database
try:
    remote_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {remote_db_name}")
except Exception as e:
    print(e)

# Connect to remote database
try:
    remote_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
    remote_conn = remote_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
remote_engine.table_names()

(pymysql.err.ProgrammingError) (1007, "Can't create database 'fantasy_project'; database exists")
[SQL: CREATE DATABASE fantasy_project]
(Background on this error at: http://sqlalche.me/e/13/f405)


['fantasy_stats',
 'qb_df_with_headshots',
 'rb_df_with_headshots',
 'super_bowl_stats',
 'te_df_with_headshots',
 'wr_df_with_headshots']

In [ ]:
rb_df_with_headshots.to_sql(name='rb_df_with_headshots', if_exists='replace', con=remote_conn, index=False)
wr_df_with_headshots.to_sql(name='wr_df_with_headshots', if_exists='replace', con=remote_conn, index=False)
te_df_with_headshots.to_sql(name='te_df_with_headshots', if_exists='replace', con=remote_conn, index=False)
qb_df_with_headshots.to_sql(name='qb_df_with_headshots', if_exists='replace', con=remote_conn, index=False)

In [9]:
#confirm tables
remote_engine.table_names()

['2021_predictions',
 'fantasy_stats',
 'qb_df_with_headshots',
 'rb_df_with_headshots',
 'super_bowl_stats',
 'te_df_with_headshots',
 'wr_df_with_headshots']

# predictions df

In [5]:
#import our CSV file
predictions_df = pd.read_csv('table.csv')
predictions_df.head()


,SEASON,Player,Position,Predicted 2021 Fantasy Points,TDS,YDS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD
0,2021,Patrick Mahomes,QB,358.63868,40.000000,4888.666667,4622.666667,38.000000,7.666667,266.000000,2.000000,0.000000,0.000000,0.000000
1,2021,Deshaun Watson,QB,334.13868,33.666667,4751.333333,4280.000000,28.333333,9.333333,469.333333,5.000000,0.333333,2.000000,0.333333
2,2021,Justin Herbert,QB,333.13868,36.000000,4570.000000,4336.000000,31.000000,10.000000,234.000000,5.000000,0.000000,0.000000,0.000000
3,2021,Kyler Murray,QB,324.01368,30.500000,4528.000000,3846.500000,23.000000,12.000000,681.500000,7.500000,0.000000,0.000000,0.000000
4,2021,Russell Wilson,QB,321.63868,37.000000,4330.000000,3923.333333,35.333333,8.333333,410.333333,1.666667,0.333333,-3.666667,0.000000


In [8]:
predictions_df.to_sql(name='2021_predictions', if_exists='replace', con=remote_conn, index=False)

In [10]:
predictions_df.columns

Index(['SEASON', 'Player', 'Position', 'Predicted 2021 Fantasy Points', 'TDS',
       'YDS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC',
       'REC YDS', 'REC TD'],
      dtype='object')